In [ ]:
pip install surprise



In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
# from surprise import KNNWithMeans
# from surprise.model_selection import GridSearchCV
from surprise.model_selection import cross_validate
from sklearn.neighbors import KDTree
from surprise import NormalPredictor



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
df = pd.read_csv("/content/drive/MyDrive/Analysis of Customer Data/ratings.csv", dtype='str')
df = df.drop(['timestamp'], axis=1)

types_dict = {'userId': int, 'movieId': int, 'rating':float}
for col, col_type in types_dict.items():
    df[col] = df[col].astype(col_type)
    


NameError: ignored

In [ ]:
def filter_values(df, column_name, threshold):
    counts = df[column_name].value_counts()
    index = counts >= threshold
    filtered_df = counts[index]
    movie_ids = filtered_df.index.tolist()
    index_filter = df['movieId'].isin(movie_ids)
    filtered_df = df[index_filter]
    print('filtered out {}% of the values'.format(100-(filtered_df.size/df.size)*100))
    return filtered_df

filtered_df = filter_values(df, 'movieId', 10)
filtered_df = filter_values(filtered_df, 'userId', 5)

filtered out 0.33258935359518205% of the values
filtered out 5.491187474239581% of the values


In [ ]:
filtered_df

,userId,movieId,rating
1,1,481,3.5
2,1,1091,1.5
3,1,1257,4.5
5,1,1590,2.5
6,1,1591,1.5
...,...,...,...
27753439,283228,8542,4.5
27753440,283228,8712,4.5
27753441,283228,34405,4.5
27753442,283228,44761,4.5


# Below the final recommender system can be found 

The below code does not contribute to the assignment directly but is an example of the results in practice. 


First we calculate average ratings for each user

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances


Mean = filtered_df.groupby(by="userId",as_index=False)['rating'].mean()
Rating_avg = pd.merge(df,Mean,on='userId')
Rating_avg['adg_rating']=Rating_avg['rating_x']-Rating_avg['rating_y']
Rating_avg.head()

,userId,movieId,rating_x,rating_y,adg_rating
0,1,307,3.5,3.269231,0.230769
1,1,481,3.5,3.269231,0.230769
2,1,1091,1.5,3.269231,-1.769231
3,1,1257,4.5,3.269231,1.230769
4,1,1449,4.5,3.269231,1.230769


Create the pivot table for each movie

In [ ]:
check = pd.pivot_table(Rating_avg[0:100000],values='rating_x',index='userId',columns='movieId')
check.head()

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,35,36,38,39,40,41,42,...,183803,183805,183815,183897,184015,184239,184253,184257,184471,184797,184937,184987,185029,185169,185989,186325,186331,186347,186587,187031,187481,187541,187593,187595,187601,187613,188213,188309,188449,189333,189363,189399,190017,190085,192005,192081,192215,192219,192225,192579
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.0,4.0,NaN,NaN,2.0,4.5,NaN,NaN,NaN,4.0,3.5,NaN,NaN,NaN,NaN,4.0,NaN,NaN,2.0,2.5,NaN,NaN,3.0,NaN,4.5,NaN,NaN,NaN,NaN,NaN,3.0,4.5,3.5,NaN,3.5,NaN,2.5,NaN,4.5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
from sklearn.metrics.pairwise import cosine_similarity
final=pd.pivot_table(Rating_avg[0:100000],values='adg_rating',index='userId',columns='movieId')

NameError: ignored

Replacing empty values by movie average

In [ ]:

final_movie = final.fillna(final.mean(axis=0))

final_user = final.apply(lambda row: row.fillna(row.mean()), axis=1)

In [ ]:

final_movie.head()

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,35,36,38,39,40,41,42,...,183803,183805,183815,183897,184015,184239,184253,184257,184471,184797,184937,184987,185029,185169,185989,186325,186331,186347,186587,187031,187481,187541,187593,187595,187601,187613,188213,188309,188449,189333,189363,189399,190017,190085,192005,192081,192215,192219,192225,192579
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.250271,-0.399359,-0.229469,-0.101467,-0.524316,0.164743,-0.456592,0.706893,-0.189628,-0.114455,0.036309,-0.77862,-0.032972,-0.251882,-0.944764,0.431480,0.164965,-0.088682,-0.835967,-0.464065,-0.033531,-0.117994,0.003844,-0.093487,0.181198,-0.327103,0.459672,-0.078011,0.459163,0.348202,-0.430466,0.328607,0.027597,-0.065294,0.306677,-0.135824,-0.048087,0.193252,0.423325,-0.791854,...,0.039625,-0.460375,-1.810526,0.966443,-0.636257,-2.310891,0.189474,0.260726,0.488444,0.448905,1.189474,-0.961988,-0.217333,0.25,0.538012,0.346304,1.189474,0.346304,0.038012,-0.619048,-0.810526,-0.208333,0.157532,-0.552469,0.488444,0.038012,0.782667,-0.461988,-0.310526,0.98607,0.986262,-0.310526,-0.142857,-0.310526,-1.065217,-0.708333,-1.708333,-2.708333,-0.208333,0.0
2,0.250271,-0.399359,-0.229469,-0.101467,-0.524316,0.164743,-0.456592,0.706893,-0.189628,-0.114455,0.036309,-0.77862,-0.032972,-0.251882,-0.944764,0.431480,0.164965,-0.088682,-0.835967,-0.464065,-0.033531,-0.117994,0.003844,-0.093487,0.181198,-0.327103,0.459672,-0.078011,0.459163,0.348202,-0.430466,0.328607,0.027597,-0.065294,0.306677,-0.135824,-0.048087,0.193252,0.423325,-0.791854,...,0.039625,-0.460375,-1.810526,0.966443,-0.636257,-2.310891,0.189474,0.260726,0.488444,0.448905,1.189474,-0.961988,-0.217333,0.25,0.538012,0.346304,1.189474,0.346304,0.038012,-0.619048,-0.810526,-0.208333,0.157532,-0.552469,0.488444,0.038012,0.782667,-0.461988,-0.310526,0.98607,0.986262,-0.310526,-0.142857,-0.310526,-1.065217,-0.708333,-1.708333,-2.708333,-0.208333,0.0
3,0.250271,-0.399359,-0.229469,-0.101467,-0.524316,0.164743,-0.456592,0.706893,-0.189628,-0.114455,0.036309,-0.77862,-0.032972,-0.251882,-0.944764,0.431480,0.164965,-0.088682,-0.835967,-0.464065,-0.033531,-0.117994,0.003844,-0.093487,0.181198,-0.327103,0.459672,-0.078011,0.459163,0.348202,-0.430466,0.328607,0.027597,-0.065294,0.306677,-0.135824,-0.048087,0.193252,0.423325,-0.791854,...,0.039625,-0.460375,-1.810526,0.966443,-0.636257,-2.310891,0.189474,0.260726,0.488444,0.448905,1.189474,-0.961988,-0.217333,0.25,0.538012,0.346304,1.189474,0.346304,0.038012,-0.619048,-0.810526,-0.208333,0.157532,-0.552469,0.488444,0.038012,0.782667,-0.461988,-0.310526,0.98607,0.986262,-0.310526,-0.142857,-0.310526,-1.065217,-0.708333,-1.708333,-2.708333,-0.208333,0.0
4,0.598551,0.598551,-0.229469,-0.101467,-1.401449,1.098551,-0.456592,0.706893,-0.189628,0.598551,0.098551,-0.77862,-0.032972,-0.251882,-0.944764,0.598551,0.164965,-0.088682,-1.401449,-0.901449,-0.033531,-0.117994,-0.401449,-0.093487,1.098551,-0.327103,0.459672,-0.078011,0.459163,0.348202,-0.401449,1.098551,0.098551,-0.065294,0.098551,-0.135824,-0.901449,0.193252,1.098551,-0.791854,...,0.039625,-0.460375,-1.810526,0.966443,-0.636257,-2.310891,0.189474,0.260726,0.488444,0.448905,1.189474,-0.961988,-0.217333,0.25,0.538012,0.346304,1.189474,0.346304,0.038012,-0.619048,-0.810526,-0.208333,0.157532,-0.552469,0.488444,0.038012,0.782667,-0.461988,-0.310526,0.98607,0.986262,-0.310526,-0.142857,-0.310526,-1.065217,-0.708333,-1.708333,-2.708333,-0.208333,0.0
5,0.250271,-0.399359,-0.229469,-0.101467,-0.524316,0.164743,-0.456592,0.706893,-0.189628,-0.114455,0.036309,-0.77862,-0.032972,-0.251882,-0.944764,0.431480,0.164965,-0.088682,-0.835967,-0.464065,-0.033531,-0.117994,0.003844,-0.093487,0.181198,-0.327103,0.459672,-0.078011,0.459163,0.348202,-0.430466,0.328607,0.027597,-0.065294,0.306677,-0.135824,-0.048087,0.193252,0.423325,-0.791854,...,0.039625,-0.460375,-1.810526,0.966443,-0.636257,-2.310891,0.189474,0.260726,0.488444,

In [ ]:
final_user.head()

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,35,36,38,39,40,41,42,...,183803,183805,183815,183897,184015,184239,184253,184257,184471,184797,184937,184987,185029,185169,185989,186325,186331,186347,186587,187031,187481,187541,187593,187595,187601,187613,188213,188309,188449,189333,189363,189399,190017,190085,192005,192081,192215,192219,192225,192579
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,...,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02,4.326923e-02
2,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,...,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16,1.480297e-16
3,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,...,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17,4.037175e-17
4,5.985507e-01,5.985507e-01,-4.030797e-03,-4.030797e-03,-1.401449e+00,1.098551e+00,-4.030797e-03,-4.030797e-03,-4.030797e-03,5.985507e-01,9.855072e-02,-4.030797e-03,-4.030797e-03,-4.030797e-03,-4.030797e-03,5.985507e-01,-4.030797e-03,-4.030797e-03,-1.401449e+00,-9.014493e-01,-4.030797e-03,-4.030797e-03,-4.014493e-01,-4.030797e-03,1.098551e+00,-4.030797e-03,-4.030797e-0

User similarity on replacing NAN by user avg

In [ ]:

b = cosine_similarity(final_user)
np.fill_diagonal(b, 0 )
similarity_with_user = pd.DataFrame(b,index=final_user.index)
similarity_with_user.columns=final_user.index
similarity_with_user.head()

userId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023,1024,1025,1026,1027,1028,1029,1030,1031,1032,1033,1034,1035,1036,1037,1038,1039,1040,1041
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.000000e+00,5.573376e-15,1.118285e-15,0.023232,-1.152350e-02,1.034012e-01,-3.748080e-15,3.212399e-01,0.0,-3.737471e-01,-1.612023e-15,4.866826e-01,-5.976918e-01,2.918404e-02,1.801448e-02,-1.701081e-01,0.00000,-4.908887e-03,0.026767,8.673617e-19,4.681937e-16,-6.338540e-01,3.999002e-01,7.112264e-01,1.084202e-18,2.693924e-02,0.000000e+00,3.937192e-01,4.774200e-15,3.331886e-15,7.391178e-02,0.000000,0.0,1.028760e-01,-1.567472e-01,-1.089939e-01,-3.781750e-01,0.025829,-1.587485e-01,-3.816136e-01,...,2.250630e-01,4.797924e-02,-6.353251e-03,-1.410145e-01,-1.723812e-01,-8.948888e-02,0.0,-8.718551e-02,6.355761e-02,-4.336809e-19,3.319523e-19,-6.787879e-01,2.039723e-15,-1.756898e-01,0.0,1.914956e-01,0.000000e+00,-3.568200e-02,1.889414e-01,-4.374558e-02,3.574038e-15,8.886188e-02,0.006927,4.197993e-15,6.414950e-01,-0.031530,-5.444442e-02,4.157366e-01,3.674683e-02,-6.212145e-01,8.673617e-19,-7.060554e-02,-6.915014e-01,1.706701e-02,4.715036e-01,-6.631967e-01,6.127490e-01,-1.679190e-01,3.788548e-02,0.530963
2,5.573376e-15,0.000000e+00,1.067786e-29,0.001087,4.219184e-02,7.986130e-03,-4.037517e-29,3.507106e-15,0.0,-2.776415e-02,-1.813910e-29,5.270016e-15,3.109606e-02,3.179511e-16,1.985338e-16,-1.861104e-15,0.00000,-5.376987e-17,0.006956,0.000000e+00,5.092861e-30,-6.933260e-15,8.816658e-02,7.779282e-15,1.540744e-32,2.149783e-16,-9.244464e-33,4.259746e-15,5.269869e-29,3.780515e-29,1.454593e-02,0.000000,0.0,1.105630e-15,-1.718047e-15,-1.175654e-15,-4.118764e-15,-0.005120,-1.735780e-15,-4.149160e-15,...,2.417424e-15,5.211518e-16,-6.842521e-17,-1.559799e-15,-1.882050e-15,-9.776353e-16,0.0,3.066157e-29,5.530985e-16,3.081488e-33,-5.748202e-33,-7.427159e-15,2.268582e-29,-1.925170e-15,0.0,2.080940e-15,0.000000e+00,-3.939624e-16,2.058147e-15,-4.796858e-16,4.019518e-29,9.723126e-16,0.005697,4.709379e-29,7.002831e-15,-0.006561,-5.544404e-16,4.515719e-15,4.036178e-16,-6.817421e-15,6.162976e-33,-7.810012e-16,-7.545241e-15,5.773816e-03,5.229003e-15,-7.284781e-15,6.740111e-15,-1.841121e-15,4.508810e-16,0.030409
3,1.118285e-15,1.067786e-29,0.000000e+00,0.032707,-2.302006e-17,1.897502e-16,-8.000319e-30,6.499635e-16,0.0,-7.491543e-16,-2.933523e-30,1.068089e-15,-1.269835e-15,4.632564e-02,4.562282e-02,-3.417991e-16,0.00000,-1.047106e-17,0.001335,0.000000e+00,1.025162e-30,-1.350262e-15,7.504082e-16,1.424051e-15,1.540744e-33,4.186340e-17,0.000000e+00,7.397208e-16,9.468991e-30,6.351324e-30,1.657553e-16,0.000000,0.0,1.928806e-16,1.060289e-02,2.156578e-02,-4.581672e-04,-0.032434,-3.377580e-16,-7.598138e-16,...,4.260507e-16,1.611245e-02,-1.191084e-17,9.099011e-03,-3.461109e-16,-1.791005e-16,0.0,5.520266e-30,5.593847e-03,0.000000e+00,-6.457578e-34,-1.364921e-15,2.738887e-02,-3.342441e-03,0.0,5.362705e-02,1.540744e-33,-7.631043e-17,2.786175e-02,-8.778694e-17,7.885864e-30,1.782259e-16,-0.003174,8.679847e-30,1.364788e-15,0.041563,-3.012665e-02,7.790314e-16,6.788967e-04,-1.327858e-15,1.540744e-33,-1.513285e-16,-1.388910e-15,3.667312e-17,1.059165e-15,-1.338447e-15,1.311470e-15,-3.379397e-16,4.423478e-03,0.011676
4,2.323228e-02,1.086643e-03,3.270651e-02,0.000000,2.329176e-02,7.458159e-02,5.108333e-02,3.622537e-02,0.0,3.023927e-02,5.157424e-02,1.989943e-02,1.011027e-02,4.836031e-02,1.817089e-02,3.049717e-02,-0.01059,1.721371e-01,0.053588,-1.642956e-02,6.373203e-02,2.299714e-02,6.691859e-02,-1.015774e-02,1.935727e-02,4.191849e-02,-5.758485e-04,-8.697175e-03,-1.406944e-02,-1.894976e-03,6.488166e-03,0.019295,0.0,2.633571e-03,4.648882e-02,4.788380e-02,1.410198e-02,-0.006806,2.098447e-02,2.024297e-02,...,3.872217e-02,2.455597e-02,5.468997e-02,1.541113e-01,1.075628e-01,-3.237088e-02

User similarity on replacing empty values by movie average

In [ ]:

cosine = cosine_similarity(final_movie)
np.fill_diagonal(cosine, 0 )
similarity_with_movie = pd.DataFrame(cosine,index=final_movie.index)
similarity_with_movie.columns=final_user.index
similarity_with_movie.head()

userId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023,1024,1025,1026,1027,1028,1029,1030,1031,1032,1033,1034,1035,1036,1037,1038,1039,1040,1041
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.000000,0.999033,0.998968,0.949255,0.997624,0.997361,0.998755,0.998254,0.999349,0.993342,0.999131,0.999044,0.998478,0.997044,0.989154,0.998406,0.999145,0.996187,0.989205,0.998915,0.998907,0.997204,0.998818,0.999103,0.998600,0.990930,0.998587,0.998155,0.998608,0.998559,0.995562,0.999013,0.999289,0.989091,0.997660,0.991283,0.997705,0.986720,0.996469,0.998501,...,0.995500,0.989875,0.996536,0.989164,0.998149,0.989676,0.999360,0.998748,0.978833,0.998796,0.999358,0.998403,0.997723,0.998631,0.999369,0.994291,0.997755,0.997569,0.988139,0.997883,0.998897,0.993482,0.976045,0.999138,0.998902,0.990058,0.974467,0.998829,0.986959,0.997823,0.999306,0.991629,0.998953,0.970081,0.998314,0.997380,0.998972,0.997310,0.978358,0.995743
2,0.999033,0.000000,0.999255,0.949529,0.997913,0.997645,0.999042,0.998540,0.999636,0.993586,0.999418,0.999331,0.998950,0.997330,0.989438,0.998692,0.999432,0.996474,0.989499,0.999202,0.999194,0.997490,0.999200,0.999390,0.998887,0.991370,0.998873,0.998442,0.998895,0.998846,0.995936,0.999300,0.999575,0.989375,0.997947,0.991568,0.997992,0.987076,0.996755,0.998787,...,0.995787,0.990159,0.996821,0.989450,0.998436,0.989961,0.999647,0.999234,0.979013,0.999083,0.999645,0.998690,0.998010,0.998917,0.999656,0.994576,0.998042,0.997856,0.988421,0.998170,0.999184,0.993767,0.976327,0.999425,0.999189,0.990396,0.974857,0.999116,0.987241,0.998110,0.999593,0.991913,0.999239,0.970423,0.998590,0.997666,0.999259,0.997597,0.978682,0.996114
3,0.998968,0.999255,0.000000,0.949732,0.997845,0.997585,0.998976,0.998475,0.999570,0.993562,0.999353,0.999265,0.998700,0.997313,0.989509,0.998627,0.999366,0.996409,0.989417,0.999136,0.999129,0.997425,0.999040,0.999324,0.998822,0.991305,0.998808,0.998376,0.998830,0.998780,0.995870,0.999235,0.999510,0.989310,0.997849,0.991589,0.997935,0.986902,0.996690,0.998722,...,0.995721,0.990086,0.996756,0.989382,0.998371,0.989896,0.999582,0.999169,0.978949,0.999017,0.999580,0.998625,0.997939,0.998842,0.999591,0.994599,0.997977,0.997791,0.988416,0.998105,0.999119,0.993702,0.976183,0.999359,0.999123,0.990467,0.974538,0.999050,0.987131,0.998044,0.999527,0.991848,0.999174,0.970298,0.998524,0.997601,0.999193,0.997531,0.978582,0.996018
4,0.949255,0.949529,0.949732,0.000000,0.947200,0.948567,0.949742,0.949241,0.949825,0.944336,0.949489,0.949663,0.949113,0.947441,0.939221,0.948673,0.949598,0.949519,0.939869,0.949500,0.949858,0.947707,0.949883,0.949784,0.949158,0.943400,0.948672,0.947913,0.949013,0.948987,0.945861,0.949326,0.949773,0.939625,0.948636,0.942769,0.947825,0.935086,0.946809,0.949140,...,0.946027,0.939341,0.947136,0.944640,0.949604,0.938102,0.949839,0.949181,0.933963,0.949453,0.949836,0.948660,0.948637,0.948680,0.949847,0.942947,0.947283,0.949372,0.939524,0.948722,0.949349,0.943034,0.925036,0.949715,0.949501,0.940313,0.927581,0.949071,0.938165,0.948505,0.949717,0.942350,0.949015,0.922515,0.948676,0.947467,0.949478,0.948025,0.926779,0.946313
5,0.997624,0.997913,0.997845,0.947200,0.000000,0.996142,0.997659,0.997131,0.998210,0.992323,0.998074,0.997875,0.997355,0.996012,0.988743,0.997288,0.998021,0.995089,0.988224,0.997792,0.997801,0.996062,0.997696,0.997979,0.997477,0.989982,0.997435,0.997091,0.997437,0.997436,0.994815,0.997890,0.998165,0.988086,0.996447,0.990299,0.996434,0.985873,0.995417,0.997378,...,0.994414,0.988726,0.995441,0.987910,0.996975,0.988984,0.998237,0.997809,0.977243,0.997670,0.998233,0.997281,0.996549,0.997493,0.998245,0.993375,0.996595,0.996452,0.986758,0.996665,0.997789,0.992317,0.974971,0.998014,0.997815,0.988910,0.973790,0.997708,0.986069,0.996707,0.998182,0.990576,0.997877,0.969532,0.997180,0.996

Create function to find neighbors

In [ ]:

def find_n_neighbors(df,n):
    order = np.argsort(df.values, axis=1)[:, :n]
    df = df.apply(lambda x: pd.Series(x.sort_values(ascending=False)
           .iloc[:n].index, 
          index=['top{}'.format(i) for i in range(1, n+1)]), axis=1)
    return df

Find the top 30 neighbors for each user 

In [ ]:

sim_user_30_u = find_n_neighbours(similarity_with_user,30)
sim_user_30_u.head()

,top1,top2,top3,top4,top5,top6,top7,top8,top9,top10,top11,top12,top13,top14,top15,top16,top17,top18,top19,top20,top21,top22,top23,top24,top25,top26,top27,top28,top29,top30
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,97,850,626,941,955,671,472,24,998,628,562,346,566,793,855,210,960,286,499,712,389,799,354,236,87,104,318,274,373,1026
2,86,685,23,429,733,84,306,695,147,498,755,5,151,227,327,82,231,605,362,13,1041,801,786,154,791,526,311,738,825,74
3,693,824,535,602,437,888,651,278,475,298,836,369,197,516,375,338,238,256,599,76,275,498,773,937,814,953,825,314,965,269
4,601,458,313,18,173,553,1005,423,180,343,953,967,593,919,830,491,719,134,163,301,627,214,435,294,402,995,185,826,1019,605
5,733,227,786,744,315,904,731,929,48,294,770,603,168,784,935,1012,588,370,421,1006,510,15,88,738,185,640,358,316,928,765


Top 30 neighbours for each user with movie


In [ ]:

sim_user_30_m = find_n_neighbours(similarity_with_movie,10)
sim_user_30_m

,top1,top2,top3,top4,top5,top6,top7,top8,top9,top10
userId,,,,,,,,,,
1,687,850,169,264,244,715,522,398,565,679
2,850,169,264,687,244,715,522,398,565,679
3,850,264,687,169,244,715,522,398,565,679
4,437,829,839,918,617,64,456,63,646,733
5,733,420,198,169,687,850,264,244,715,522
...,...,...,...,...,...,...,...,...,...,...
1037,756,832,687,169,264,850,244,522,398,679
1038,144,266,576,687,850,169,264,244,715,522
1039,541,474,169,264,850,687,244,715,522,344


Create function to find similar movies

In [ ]:
def get_user_similar_movies( user1, user2 ):
    common_movies = Rating_avg[Rating_avg.userId == user1].merge(
    Rating_avg[Rating_avg.userId == user2],
    on = "movieId",
    how = "inner" )
    return common_movies

In [ ]:

a = get_user_similar_movies(370,86309)
a = a.loc[ : , ['rating_x_x','rating_x_y','movieId']]
a.head()

,rating_x_x,rating_x_y,movieId
0,2.0,3.5,356
1,3.0,3.0,380
2,3.0,2.5,480


# TODO: 
- Apply cross validation
- Get RMSE
- apply gridsearch as project description says
